In [10]:
!ls ../../papers_drug_discovery/'2 Computational Review Papers'/'2007 The role of multiscale computational approaches.pdf'

'../../papers_drug_discovery/2 Computational Review Papers/2007 The role of multiscale computational approaches.pdf'


In [235]:
!ls ../../papers_HPC/2407.04716v1.pdf

../../papers_HPC/2407.04716v1.pdf


In [11]:
! pip install pymupdf


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import fitz

In [13]:
from PIL import Image

In [187]:
import pymupdf

pdf = pymupdf.open("../../papers_drug_discovery/2 Computational Review Papers/2007 The role of multiscale computational approaches.pdf")
#pdf = pymupdf.open("../../papers_HPC/2407.04716v1.pdf")
page = pdf[3]

print("mediabox", page.mediabox)
print("cropbox", page.cropbox)

mediabox Rect(0.0, 0.0, 612.0, 792.0)
cropbox Rect(0.0, 0.0, 612.0, 792.0)


## Page Dict

In [188]:

doc_order = []

# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = page.get_text("dict")["blocks"]
for b in blocks:  # iterate through the text blocks
    #print(b)
    if "ext" in b:
        if b["ext"] == "jpeg" or b["ext"] == "png":
            doc_order.append(["image", b["bbox"][0], b["bbox"][1], b["bbox"][2], b["bbox"][3]])
        #else:
            #print(b)
    elif "lines" in b:
        #print(b["bbox"])
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                doc_order.append(["text", s["bbox"][0], s["bbox"][1], s["bbox"][2], s["bbox"][3]])
    #else:
        #print(b)

#for d in doc_order:
#    print(d)
            

In [189]:
# What range of coordinates does text content take up?
minx = doc_order[0][1]
miny = doc_order[0][2]
maxx = doc_order[0][1]
maxy = doc_order[0][2]

for d in doc_order:
    if d[1] < minx:
        minx = d[1]

    if d[1] > maxx:
        maxx = d[1]

    if d[2] < miny:
        miny = d[2]

    if d[1] > maxy:
        maxy = d[2]

    if d[3] < minx:
        minx = d[3]

    if d[3] > maxx:
        maxx = d[3]

    if d[4] < miny:
        miny = d[4]

    if d[4] > maxy:
        maxy = d[4]

print(minx, miny, maxx, maxy)

53.849998474121094 28.74383544921875 543.7561645507812 762.0636596679688


## Annotations

In [190]:
page_content = page

In [191]:
def two_lines_overlap(shape_items):
    abs(shape_items[0][1] - shape_items[1][1]) < 2

def two_lines_cross(shape_items):
    print(shape_items)
    print(shape_items[0])
    print(shape_items[0][1])
    
    l1y1 = shape_items[0][1][1]
    l1y2 = shape_items[0][2][1]
    if l1y1 > l1y2:
        l1maxy = l1y1
        l1miny = l1y2
    else:
        l1maxy = l1y2
        l1miny = l1y1

    l2y1 = shape_items[1][1][1]
    l2y2 = shape_items[1][2][1]
    if l1y1 > l1y2:
        l2maxy = l1y1
        l2miny = l1y2
    else:
        l2maxy = l1y2
        l2miny = l1y1

    if l1maxy > l2miny + 1:
        if l1miny < l2maxy - 1:
            print("cross")
            return True

    print(" no cross")
    return False
    

In [192]:
def remove_shape(shape_type, shape_rect, shape_items):
    if shape_type == "s":

        if len(shape_items) == 1:
            return True

        if len(shape_items) == 2:
            if two_lines_overlap(shape_items):
                return True
            elif two_lines_cross(shape_items):
                return False
            else: # they must be parallel
                return True

    if shape_type == "f":
        # if it is basically a line
        print("f", abs(shape_rect[1] - shape_rect[3]))
        if abs(shape_rect[1] - shape_rect[3]) < 2:
            return True
    

In [193]:
def insert_in_doc_order(shape):
    i = 0
    not_found = True
    while i < len(doc_order) and not_found:
        #print(i, doc_order[i][0],shape[0])
        if doc_order[i][2] > shape[2]: # comparing on y coordinate
            #print(i, "inserted")
            doc_order.insert(i, shape)
            not_found = False
        i = i + 1

In [194]:
from pymupdf import Rect, Point

newdoc = pymupdf.open()
page = newdoc.new_page()
nshape = page.new_shape()

i = 0
for shape in page_content.get_drawings():
    print(i)
    i = i + 1
    #if shape["type"] == "s":
#    print("shape", shape["type"])
#    print(shape["items"])
#    print(shape["rect"])
    if not remove_shape(shape["type"], shape["rect"], shape["items"]):
        insert_in_doc_order(["shape" + shape["type"],  shape["rect"][0],  shape["rect"][1],  shape["rect"][2],  shape["rect"][3]])
        for item in shape['items']:
            if item[0] == 're':
                nshape.draw_rect(item[1])
            elif item[0] == 'l':
                nshape.draw_line(item[1], item[2])
            elif item[0] == 'c':
                nshape.draw_bezier(item[1], item[2], item[3], item[4])

#        else:
#            print("------------unknown-------------", ["shape",  shape["type"], shape["rect"][1], shape["items"], shape["rect"]])
    
        #print(shape['type'], shape['fill_opacity'], shape['fill'],
        #      shape['type'], shape['stroke_opacity'], shape['color'], shape['width'], shape['lineCap'], 
        #      shape['lineJoin'], shape['closePath'], shape['dashes'], shape['layer'], 
        #      shape['seqno'], shape['fill'], shape['fill_opacity'], shape['even_odd'])
        #nshape.finish(width = shape['width'], fill=shape['fill'])
        #shape_details = {'fill': shape['fill']}
    nshape.finish(fill=shape['fill'])

nshape.commit(overlay=True)
newdoc.save("output.pdf")

for page in newdoc:
    pix = page.get_pixmap()
    pix.save("newdoc.png")

0
[('l', Point(53.849998474121094, 749.5499877929688), Point(541.1500244140625, 749.5499877929688)), ('l', Point(53.849998474121094, 39.95001220703125), Point(541.1500244140625, 39.95001220703125))]
('l', Point(53.849998474121094, 749.5499877929688), Point(541.1500244140625, 749.5499877929688))
Point(53.849998474121094, 749.5499877929688)
 no cross
1
2
3
4
5
6
7
8
9
10
11
12
13
f 7.68096923828125
14
[('l', Point(201.91600036621094, 111.36199951171875), Point(254.24899291992188, 284.36199951171875)), ('l', Point(254.24899291992188, 284.36199951171875), Point(273.9159851074219, 306.36199951171875))]
('l', Point(201.91600036621094, 111.36199951171875), Point(254.24899291992188, 284.36199951171875))
Point(201.91600036621094, 111.36199951171875)
cross
15
f 8.587005615234375
16
17
f 5.6099853515625
18
f 5.930999755859375
19
f 5.6099853515625
20
f 5.6099853515625
21
f 5.6099853515625
22
23
f 5.5260009765625
24
25
f 5.5269775390625
26
27
28
29
30
31
32
33
34
f 2.0
35
36
37
f 2.0
38
f 2.0


In [195]:
import pandas as pd

In [196]:
df = pd.DataFrame(doc_order, columns = ['type', 'x0', 'y0', 'x1', 'y1'])

In [197]:
df = df.sort_values(by='y0')

In [198]:
from tabulate import tabulate

In [199]:
print(tabulate(df, headers = 'keys', tablefmt = 'psql'))

+-----+---------+----------+----------+----------+----------+
|     | type    |       x0 |       y0 |       x1 |       y1 |
|-----+---------+----------+----------+----------+----------|
|  37 | text    |  53.85   |  28.7438 | 125.462  |  38.0212 |
|  93 | text    | 253.914  |  51.6882 | 481.078  |  67.3288 |
|  92 | text    |  82.2498 |  59.0222 | 216.832  |  74.6628 |
|   0 | shapefs | 251.916  |  67.028  | 488.583  |  67.028  |
|   1 | shapefs | 234.249  |  69.029  | 512.583  | 374.695  |
|   2 | shapefs | 278.814  |  74.529  | 425.993  | 100.029  |
|   3 | shapefs |  81.75   |  76.279  | 214.499  | 352.778  |
|  94 | text    | 298.583  |  80.8323 | 408.625  |  94.2386 |
|   4 | shapef  | 276.412  |  84.463  | 285.896  |  90.073  |
|   5 | shapef  | 276.412  |  84.463  | 285.896  |  90.073  |
|   6 | shapef  | 276.412  |  84.463  | 285.896  |  90.073  |
|   7 | shapef  | 276.412  |  84.463  | 285.896  |  90.073  |
|   8 | shapefs | 455.749  |  86.029  | 508.749  | 364.028  |
|   9 | 

In [200]:
df = df[df['type'].str.contains("shape")]

In [201]:
print(tabulate(df, headers = 'keys', tablefmt = 'psql'))

+----+---------+---------+---------+---------+---------+
|    | type    |      x0 |      y0 |      x1 |      y1 |
|----+---------+---------+---------+---------+---------|
|  0 | shapefs | 251.916 |  67.028 | 488.583 |  67.028 |
|  1 | shapefs | 234.249 |  69.029 | 512.583 | 374.695 |
|  2 | shapefs | 278.814 |  74.529 | 425.993 | 100.029 |
|  3 | shapefs |  81.75  |  76.279 | 214.499 | 352.778 |
|  4 | shapef  | 276.412 |  84.463 | 285.896 |  90.073 |
|  5 | shapef  | 276.412 |  84.463 | 285.896 |  90.073 |
|  6 | shapef  | 276.412 |  84.463 | 285.896 |  90.073 |
|  7 | shapef  | 276.412 |  84.463 | 285.896 |  90.073 |
|  8 | shapefs | 455.749 |  86.029 | 508.749 | 364.028 |
|  9 | shapes  | 202.583 |  86.695 | 279.583 | 307.695 |
| 10 | shapefs |  92.083 |  88.696 | 202.416 | 133.029 |
| 11 | shapefs | 251.249 |  99.529 | 453.749 | 169.528 |
| 12 | shapef  | 256.472 | 103.195 | 258.36  | 105.195 |
| 13 | shapes  | 202.249 | 108.362 | 266.583 | 194.362 |
| 14 | shapes  | 201.916 | 111.

In [202]:
minx = df['x0'].min()
miny = df['y0'].min()
maxx = df['x1'].max()
maxy = df['y1'].max()

In [203]:
print(minx, miny, maxx, maxy)

81.75 67.02801513671875 512.5830078125 374.69500732421875


In [204]:
mediabox = page.mediabox

In [205]:
pagex0 = mediabox[0]
pagey0 = mediabox[1]
pagex1 = mediabox[2]
pagey1 = mediabox[3]

print(pagex0, pagey0, pagex1, pagey1)

lenx = pagex1 - pagex0
leny = pagey1 - pagey0

print(lenx, leny)

0.0 0.0 595.0 842.0
595.0 842.0


In [206]:
minxpercent = minx / lenx
minypercent = miny / leny
maxxpercent = maxx / lenx
maxypercent = maxy / leny

print(minxpercent, minypercent, maxxpercent, maxypercent)

0.1373949579831933 0.07960571868968973 0.8614840467436975 0.44500594694087736


In [207]:
! pip install pdf2image


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [208]:
from pdf2image import convert_from_path

In [209]:
images = convert_from_path("../../papers_drug_discovery/2 Computational Review Papers/2007 The role of multiscale computational approaches.pdf")

In [210]:
#images = convert_from_path("../../papers_HPC/2407.04716v1.pdf")

In [211]:
images[3].save('page1' +'.jpg', 'JPEG')

In [212]:
# Importing Image class from PIL module
from PIL import Image
 
# Opens a image in RGB mode
im = Image.open('page1' +'.jpg')

In [213]:
width, height = im.size

In [214]:
print(width, height)

1700 2200


In [215]:
left = (minxpercent - 0.05) * width
top = (minypercent - 0.05) * height
right = (maxxpercent + 0.05) * width
bottom = (maxypercent + 0.06) * height

In [216]:
im1 = im.crop((left, top, right, bottom))

In [217]:
im1.show()

In [218]:
im1.save('im1' +'.jpg', 'JPEG')